#CafChem tools docking and rescoring with the UMA MLIP

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MauricioCafiero/Research_Notebooks/blob/main/Rescore_Docking_UMA_CafChem.ipynb)

## This notebook allows you to:
- dock a single SMILES string, a list of string, or a CSV file with SMILES in one column.
- save poses as SDF files.
- Calculate the interaction between the ligand and the protein using Meta's UMA MLIP

## Requirements:
- This notebook will install deepchem, dockstring, openBabel, Fairchem and py3Dmol
- It will pull the CafChem tools from Github.
- It will install all needed libraries.
- You need to have a HF_Token set as a secret to access the UMA MLIP.

# set-up

This block:

- Loads all needed modules/libraries
    

    


### Install a few libraries

In [1]:
! pip install deepchem
! pip install dockstring
! pip install openbabel-wheel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 67.8 MB/s eta 0:00:00


In [2]:
! pip install py3Dmol
! pip install fairchem-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.9/284.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.0/448.0 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 806.9 kB/s eta 0:00:00
   ━━━━━

### Import libraries, pull CafChem from Github

In [15]:
!git clone https://github.com/MauricioCafiero/CafChem.git

Cloning into 'CafChem'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 23 (delta 7), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (23/23), 18.57 KiB | 4.64 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [16]:
import numpy as np
import pandas as pd
import deepchem as dc
import time
from rdkit import Chem
import matplotlib.pyplot as plt
from rdkit.Chem import AllChem, Draw
from deepchem.feat.smiles_tokenizer import SmilesTokenizer
from dockstring import load_target
from google.colab import files
import os
import py3Dmol
import ase.io
from ase import Atoms
from fairchem.core import FAIRChemCalculator, pretrained_mlip
import CafChem.CafChemReDock as ccr

cpuCount = os.cpu_count()
print(cpuCount)

2


## Set-up Fairchem
- Must have HF_TOKEN saved as a secret

In [5]:
predictor = pretrained_mlip.get_predict_unit("uma-s-1", device="cpu")
calculator = FAIRChemCalculator(predictor, task_name="omol")
model = "UMA-OMOL"

checkpoints/uma-s-1.pt:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

# Calculations

## Dock molecules
- tools available include ccr.dock_dataframe, ccr.dock_list and ccr.dock_smiles
- for each you must supply as arguements the SMILES input (either a filename, a list, or a SMILES string), the target protein, and the number of CPU cores to use. For ccr.dock_dataframe, you must also provide the key for the SMILES column in the CSV file.
- xyz structures can be visualized via the ccr.visualize_molecule tool. This accepts an XYZ string as an arguement. This may be easily extracted from an XYZ file as seen below.

In [6]:
ccr.dock_dataframe("file.csv","HMGCR",cpuCount, "smiles",)

Docking 1 molecules in HMGCR.
Docking molecule 1.
SDF file written for score -4.5


In [7]:
df = pd.read_csv("file.csv")
smiles_list = df["smiles"].tolist()
ccr.dock_list(smiles_list,"HMGCR",cpuCount)

Docking 1 molecules in HMGCR.
Docking molecule 1.
SDF file written for score -4.5


In [18]:
statin = "OC(=O)C[C@H](O)C[C@H](O)\C=C\c1c(C(C)C)nc(N(C)S(=O)(=O)C)nc1c2ccc(F)cc2"
ccr.dock_smiles(statin,"HMGCR",cpuCount)

Docking molecule.
SDF file written for score -8.1


## Calculate interaction energies between a docking pose and the protein using Meta's UMA MLIP
- If CafChem has an XYZ QM active site pepared for the protein, then the interaction between a ligand (SDF file) and the protein active site (from the library) may be calculated using Meta's UMA MLIP.
- supply as arguements the name of the SDF file (without .sdf), the protein information (in the form ccr.[your protein]_data), the ASE calculator, ans the charge and spin multiplicty of the ligand.
- returns a list of XYZ strings for the ligands in the input SDF files.
- the XYZ strings may be visualized with the ccr.visualize_molecule tool, which accepts as its arguement the XYZ string.

In [19]:
total_xyz = ccr.uma_interaction("trial_1", ccr.HMGCR_data, calculator, -1, 1)

Energy of ligand is: -1968.269 kcal/mol
Energy of active site is: -7759.170 kcal/mol
The size of the complex is: 391
Energy of complex is: -9727.851 kcal/mol
Energy difference is: -0.412 kcal/mol


In [20]:
ccr.visualize_molecule(total_xyz[0])

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [21]:
f = open(ccr.HMGCR_data["file_location"],"r")
structure = f.read()
f.close()

ccr.visualize_molecule(structure)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.